In [1]:
from transformers import AutoModelForQuestionAnswering, AutoModel, AutoConfig, get_linear_schedule_with_warmup
from transformers.optimization import AdamW
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from pathlib import Path
import os
from itertools import compress
import utils

In [2]:
from torch.utils.data import DataLoader
from apex.optimizers.fused_lamb import FusedLAMB as Lamb
from fairseq import criterions 
from functools import partial
from tokenizers import BertWordPieceTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from fastai.core import *
from fastai.text import *

In [3]:
path = Path.cwd()
file_dir = Path.cwd()/'data'

In [4]:
train_df = pd.read_csv(file_dir/'train.csv')
train_df['text'] = train_df['text'].apply(lambda x: str(x))
train_df['sentiment'] = train_df['sentiment'].apply(lambda x: str(x))
train_df['selected_text'] = train_df['selected_text'].apply(lambda x: str(x))

In [5]:
fold_df = pd.read_csv(file_dir/'train_folds.csv')
fold_df['text'] = fold_df['text'].apply(lambda x: str(x))
fold_df['sentiment'] = fold_df['sentiment'].apply(lambda x: str(x))
fold_df['selected_text'] = fold_df['selected_text'].apply(lambda x: str(x))

In [6]:
test_df = pd.read_csv(file_dir/'test.csv')
test_df['text'] = test_df['text'].apply(lambda x: str(x))
test_df['sentiment'] = test_df['sentiment'].apply(lambda x: str(x))

In [7]:
max_len = 128
bs = 32
tokenizer = BertWordPieceTokenizer('input/electra-base-disc/vocab.txt', lowercase=True)

In [8]:
def preprocess(sentiment, tweet, selected, tokenizer, max_len):
    _input = tokenizer.encode(sentiment, tweet)
    _span = tokenizer.encode(selected, add_special_tokens=False)
    
    len_span = len(_span.ids)
    start_idx = None
    end_idx = None
    
    for ind in (i for i, e in enumerate(_input.ids) if e == _span.ids[0]):
        if _input.ids[ind: ind + len_span] == _span.ids:
            start_idx = ind
            end_idx = ind + len_span - 1
            break
    
    # Handles cases where Wordpiece tokenizing input & span separately produces different outputs
    if not start_idx:
        idx0 = tweet.find(selected)
        idx1 = idx0 + len(selected)
        
        char_targets = [0] * len(tweet)
        if idx0 != None and idx1 != None:
            for ct in range(idx0, idx1):
                char_targets[ct] = 1
                
        tweet_offsets = list(compress(_input.offsets, _input.type_ids))[0:-1]
        
        target_idx = []
        for j, (offset1, offset2) in enumerate(tweet_offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)
                
        start_idx, end_idx = target_idx[0] +3 , target_idx[-1] + 3
        
    _input.start_target = start_idx
    _input.end_target = end_idx
    _input.tweet = tweet
    _input.sentiment = sentiment
    _input.selected = selected
    
    _input.pad(max_len)
    
    return _input

In [9]:
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss    

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, ε:float=0.1, reduction='mean'):
        super().__init__()
        self.ε,self.reduction = ε,reduction
    
    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return torch.lerp(nll, loss/c, self.ε) 

In [11]:
class TweetDataset(Dataset):
    def __init__(self, dataset, test = None):
        self.df = dataset
        self.test = test
        
    def __getitem__(self, idx):
        if not self.test:
            sentiment, tweet, selected = (self.df[col][idx] for col in ['sentiment', 'text', 'selected_text'])
            _input = preprocess(sentiment, tweet, selected, tokenizer, max_len)
            
            yb = [torch.tensor(_input.start_target), torch.tensor(_input.end_target)]
            
        else:
            _input = tokenizer.encode(self.df.sentiment[idx], self.df.text[idx])
            _input.pad(max_len)
            
            yb = 0

        xb = [torch.LongTensor(_input.ids),
              torch.LongTensor(_input.attention_mask),
              torch.LongTensor(_input.type_ids),
              np.array(_input.offsets)]

        return xb, yb     

    def __len__(self):
        return len(self.df)

In [8]:
# config = AutoConfig.from_pretrained('google/electra-base-discriminator')
# config.output_hidden_states = True

# pt_model = AutoModel.from_pretrained('google/electra-base-discriminator', config = config)
# pt_model.save_pretrained(electra_path)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/google/electra-base-discriminator/config.json from cache at /home/jack/.cache/torch/transformers/9236d197566a7f1be2b2151f5afcc5a8e17f31e1e23c52f3cdf2340019986e78.88ba6e8e7d5a7936e86d6f2551fe19c236dc57c24da163907cd0544e9933f6ee
INFO:transformers.configuration_utils:Model config ElectraConfig {
  "_num_labels": 2,
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embedding_size": 768,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    

In [12]:
electra_path = path/'input/electra-base-disc'
pt_model = AutoModel.from_pretrained(electra_path)

INFO:transformers.configuration_utils:loading configuration file /home/jack/Documents/DL/kaggle/tweet_qa/input/electra-base-disc/config.json
INFO:transformers.configuration_utils:Model config ElectraConfig {
  "_num_labels": 2,
  "architectures": [
    "ElectraModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embedding_size": 768,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "electra",
  "no_repeat_ngram_size"

In [13]:
class SpanModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = pt_model
        self.drop_out = nn.Dropout(0.1)
        self.qa_outputs = nn.Linear(768 * 2, 2) # update hidden size

    # could pass offsets here and not use - can grab in last_input
    def forward(self, input_ids, attention_mask, token_type_ids, offsets = None):
        
        _, hidden_states = self.model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        out = torch.cat((hidden_states[-1], hidden_states[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.qa_outputs(out)
        
        start_logits, end_logits = logits.split(1, dim=-1)
        
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [14]:
class CELoss(Module):
    def __init__(self, loss_fn = nn.CrossEntropyLoss()): 
        self.loss_fn = loss_fn
        
    def forward(self, inputs, start_targets, end_targets):
        start_logits, end_logits = inputs # assumes tuple input
        
        logits = torch.cat([start_logits, end_logits]).contiguous()
        
        targets = torch.cat([start_targets, end_targets]).contiguous()
        
        return self.loss_fn(logits, targets)

In [15]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [16]:
# Note that validation ds is by default not shuffled in fastai - so indexing like this will work for Callback
# https://forums.fast.ai/t/how-to-set-shuffle-false-of-train-and-val/33730

class JaccardScore(Callback):
    "Stores predictions and targets to perform calculations on epoch end."
    def __init__(self, valid_ds): 
        self.valid_ds = valid_ds
        self.context_text = valid_ds.df.text
        self.answer_text = valid_ds.df.selected_text
        
    def on_epoch_begin(self, **kwargs):
        self.jaccard_scores = []  
        self.valid_ds_idx = 0
        
        
    def on_batch_end(self, last_input:Tensor, last_output:Tensor, last_target:Tensor, **kwargs):
              
        input_ids = last_input[0]
        attention_masks = last_input[1].bool()
        token_type_ids = last_input[2].bool()
        offsets = last_input[3]

        start_logits, end_logits = last_output
        
        # for id in batch of ids
        for i in range(len(input_ids)):
            
            _offsets = offsets[i]
            start_idx, end_idx = torch.argmax(start_logits[i]), torch.argmax(end_logits[i])
            _answer_text = self.answer_text[self.valid_ds_idx]
            original_start, original_end = _offsets[start_idx][0], _offsets[end_idx][1]
            pred_span = self.context_text[self.valid_ds_idx][original_start : original_end]
                
            score = jaccard(pred_span, _answer_text)
            self.jaccard_scores.append(score)

            self.valid_ds_idx += 1
            
    def on_epoch_end(self, last_metrics, **kwargs):        
        res = np.mean(self.jaccard_scores)
        return add_metrics(last_metrics, res)

In [17]:
from fastai.callbacks import *

def new_on_train_begin(self, **kwargs:Any)->None:
    "Initializes the best value."
    if not hasattr(self, 'best'):
        self.best = float('inf') if self.operator == np.less else -float('inf')

SaveModelCallback.on_train_begin = new_on_train_begin

In [18]:
def run_fold(fold):
    n_epochs = 5
    wd = 0.01 # high value but worked quite well
    lr = 3e-5
    
    tr_df  = fold_df[fold_df.kfold != fold].reset_index(drop=True)
    val_df = fold_df[fold_df.kfold == fold].reset_index(drop=True)
    
    train_ds, valid_ds = [TweetDataset(i) for i in [tr_df,val_df]]
    data = DataBunch.create(train_ds, valid_ds,bs = bs)
    loss_fn = partial(CELoss, LabelSmoothingCrossEntropy())

    model = SpanModel()
    learner = Learner(data, model, loss_func = loss_fn(), opt_func = Lamb, metrics = [JaccardScore(valid_ds)])
    
    early_stop_cb = EarlyStoppingCallback(learner, monitor='jaccard_score',mode='max',patience=2)
    save_model_cb = SaveModelCallback(learner,every='improvement',monitor='jaccard_score',name=f'electra-qa-fold_{fold}')
    csv_logger_cb = CSVLogger(learner, f"training_logs_{fold}", True)
    

    
    learner.fit_one_cycle(n_epochs, lr, pct_start=0.4, div_factor=50, 
                          wd=wd, callbacks=[early_stop_cb, save_model_cb, csv_logger_cb])
    
    learner.save(f'electra-qa-finetune_{fold}', with_opt=False)
    del learner; gc.collect()

In [20]:
for fold in range(1,5): run_fold(fold)

epoch,train_loss,valid_loss,jaccard_score,time
0,2.753928,2.432873,0.588696,08:51
1,1.755396,1.720233,0.666024,08:51
2,1.647031,1.636848,0.684407,08:54
3,1.592381,1.623212,0.686574,08:55
4,1.556036,1.627867,0.685629,08:52


Better model found at epoch 0 with jaccard_score value: 0.588696261312984.
Better model found at epoch 1 with jaccard_score value: 0.6660237120058567.
Better model found at epoch 2 with jaccard_score value: 0.6844068907744483.
Better model found at epoch 3 with jaccard_score value: 0.6865743327123112.


epoch,train_loss,valid_loss,jaccard_score,time
0,1.970680,1.680591,0.669833,08:56
1,1.595243,1.547934,0.709651,09:02
2,1.562007,1.533304,0.715251,08:54
3,1.520661,1.529326,0.716113,08:53
4,1.468122,1.529801,0.714205,08:57


Better model found at epoch 0 with jaccard_score value: 0.6698327268562921.
Better model found at epoch 1 with jaccard_score value: 0.7096507222209028.
Better model found at epoch 2 with jaccard_score value: 0.7152511171290885.
Better model found at epoch 3 with jaccard_score value: 0.7161134023574971.


epoch,train_loss,valid_loss,jaccard_score,time
0,1.745695,1.550403,0.717438,08:55
1,1.543549,1.487153,0.727698,09:02
2,1.523964,1.483014,0.736880,08:59
3,1.451921,1.473427,0.736957,08:47
4,1.449583,1.474274,0.739977,08:45


Better model found at epoch 0 with jaccard_score value: 0.7174378378907009.
Better model found at epoch 1 with jaccard_score value: 0.7276977379545337.
Better model found at epoch 2 with jaccard_score value: 0.736880033296791.
Better model found at epoch 3 with jaccard_score value: 0.7369571472549806.
Better model found at epoch 4 with jaccard_score value: 0.7399766715701668.


epoch,train_loss,valid_loss,jaccard_score,time
0,1.750307,1.489576,0.745879,08:45
1,1.482321,1.407582,0.766283,08:45
2,1.429331,1.398167,0.768385,08:45
3,1.406674,1.388414,0.769556,08:45
4,1.391588,1.387487,0.771310,08:45


Better model found at epoch 0 with jaccard_score value: 0.7458792329666973.
Better model found at epoch 1 with jaccard_score value: 0.7662826493425285.
Better model found at epoch 2 with jaccard_score value: 0.768384986278449.
Better model found at epoch 3 with jaccard_score value: 0.7695562016141507.
Better model found at epoch 4 with jaccard_score value: 0.7713103679410099.


### Predict

In [20]:
model = SpanModel()
tr_df  = fold_df[fold_df.kfold != 0].reset_index(drop=True)
val_df = fold_df[fold_df.kfold == 0].reset_index(drop=True)
train_ds, valid_ds = [TweetDataset(i) for i in [tr_df, val_df]]
test_ds = TweetDataset(test_df, test = True)
loss_fn = partial(CELoss, LabelSmoothingCrossEntropy())

data = DataBunch.create(train_ds, valid_ds, test_ds, path=".", bs = bs)
learner = Learner(data, model, loss_func = loss_fn(), path = path/'models', model_dir=f".")

In [90]:
preds = []
test_df_idx = 0

with torch.no_grad():
    
    for xb,yb in tqdm(learner.data.test_dl):
        model0 = learner.load(f'electra-qa-finetune_0').model.eval()
        start_logits0, end_logits0 = to_cpu(model0(*xb))
        start_logits0, end_logits0 = start_logits0.float(), end_logits0.float()
        
        model1 = learner.load(f'electra-qa-finetune_1').model.eval()
        start_logits1, end_logits1 = to_cpu(model1(*xb))
        start_logits1, end_logits1 = start_logits1.float(), end_logits1.float()
        
        model2 = learner.load(f'electra-qa-finetune_2').model.eval()
        start_logits2, end_logits2 = to_cpu(model2(*xb))
        start_logits2, end_logits2 = start_logits2.float(), end_logits2.float()
        
        model3 = learner.load(f'electra-qa-finetune_3').model.eval()
        start_logits3, end_logits3 = to_cpu(model3(*xb))
        start_logits3, end_logits3 = start_logits3.float(), end_logits3.float()
        
        model4 = learner.load(f'electra-qa-finetune_4').model.eval()
        start_logits4, end_logits4 = to_cpu(model4(*xb))
        start_logits4, end_logits4 = start_logits4.float(), end_logits4.float()
        
        input_ids = to_cpu(xb[0])
        attention_masks = to_cpu(xb[1].bool())
        token_type_ids = to_cpu(xb[2].bool())
        offsets = to_cpu(xb[3])
        
        start_logits = (start_logits0 + start_logits1 + start_logits2 + start_logits3 + start_logits4) / 5
        end_logits = (end_logits0 + end_logits1 + end_logits2 + end_logits3 + end_logits4) / 5
        
        for i in range(len(input_ids)):
            
            _offsets = offsets[i]
            start_idx, end_idx = torch.argmax(start_logits[i]), torch.argmax(end_logits[i])
            original_start, original_end = _offsets[start_idx][0], _offsets[end_idx][1]
            pred_span = test_ds.df.text[test_df_idx][original_start : original_end]
            preds.append(pred_span)
            test_df_idx += 1

100%|██████████| 111/111 [04:38<00:00,  2.51s/it]


In [108]:
out_df = test_df

In [110]:
out_df['selected_text'] = preds
out_df['selected_text'] = out_df.apply(lambda o: o['text'] if len(o['text']) < 3 else o['selected_text'], 1)
out_df['selected_text'] = out_df.apply(lambda o: o['text'] if o['sentiment'] == 'neutral' else o['selected_text'], 1)

In [117]:
subdf = out_df[['textID', 'selected_text']]
subdf.to_csv("submission1.csv", index=False)